In [ ]:
#fl studio
#mac 
#ejecutble 


!pip install pydub --upgrade
from pydub import AudioSegment
from google.colab import files

# Definir las variables del compresor y del intervalo de tiempo
threshold = -20.0  # umbral en dB
ratio = 4.0  # relación de compresión
output_gain = 6.0  # ganancia de salida en dB
start_min = 1.0  # inicio del intervalo de tiempo en minutos
end_min = 2.5  # final del intervalo de tiempo en minutos

# Cargar el archivo de audio
uploaded = files.upload()
audio_file = AudioSegment.from_file(next(iter(uploaded)), format="mp3")

# Convertir los minutos a muestras de audio
start = int(start_min * 60 * 1000)  # tiempo de inicio en milisegundos
end = int(end_min * 60 * 1000)  # tiempo final en milisegundos

# Seleccionar solo el segmento de audio correspondiente al intervalo de tiempo
audio_segment = audio_file[start:end]

# Aplicar la compresión con las variables definidas
#compressed_audio = audio_segment.compress_dynamic_range(threshold=threshold, ratio=ratio, attack=10, release=50, output_gain=output_gain)
compressed_audio = audio_segment.compress_dynamic_range(threshold=threshold, ratio=ratio, attack=10, release=50)

# Cambiar output_gain a -10 dB
compressed_audio = compressed_audio.apply_gain(output_gain)


# Agregar fade in/out
compressed_audio = compressed_audio.fade_in(2000).fade_out(3000)

# Cambiar la velocidad
compressed_audio = compressed_audio.speedup(playback_speed=1.5)

# Cambiar el tono
#compressed_audio = compressed_audio.pitch_shift(5)


# Crear un segmento de audio vacío de 60 segundos
silence = AudioSegment.empty()
silence += AudioSegment.silent(duration=60000)

# Superponer el segmento vacío en el minuto deseado
compressed_audio = compressed_audio.overlay(silence, position=60000)


# Guardar el archivo comprimido
with open("output_audio.mp3", "wb") as f:
    f.write(compressed_audio.export(format="mp3").read())



files.download("output_audio.wav")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving 520089085.mp3 to 520089085 (1).mp3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install PyQt5
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QSlider, QLabel, QVBoxLayout
from PyQt5.QtCore import Qt
from pydub import AudioSegment
from pydub.playback import play

class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        # Cargar la señal de audio
        self.audio_file = AudioSegment.from_file("mi_archivo_de_audio.mp3", format="mp3")
        
        # Crear controles deslizantes para cada banda de frecuencia
        self.slider_low = QSlider(Qt.Horizontal)
        self.slider_low.setMinimum(-12)
        self.slider_low.setMaximum(12)
        self.slider_low.setValue(0)
        
        self.slider_mid = QSlider(Qt.Horizontal)
        self.slider_mid.setMinimum(-12)
        self.slider_mid.setMaximum(12)
        self.slider_mid.setValue(0)
        
        self.slider_high = QSlider(Qt.Horizontal)
        self.slider_high.setMinimum(-12)
        self.slider_high.setMaximum(12)
        self.slider_high.setValue(0)
        
        # Crear etiquetas para mostrar los valores de cada banda de frecuencia
        self.label_low = QLabel("Baja frecuencia: 0 dB")
        self.label_mid = QLabel("Frecuencia media: 0 dB")
        self.label_high = QLabel("Alta frecuencia: 0 dB")
        
        # Conectar los controles deslizantes a los métodos de actualización de etiquetas
        self.slider_low.valueChanged.connect(self.update_label_low)
        self.slider_mid.valueChanged.connect(self.update_label_mid)
        self.slider_high.valueChanged.connect(self.update_label_high)
        
        # Crear un botón para aplicar los cambios de ecualización
        self.button_apply = QPushButton("Aplicar")
        self.button_apply.clicked.connect(self.apply_eq)
        
        # Crear un diseño vertical para la GUI
        vbox = QVBoxLayout()
        vbox.addWidget(self.slider_low)
        vbox.addWidget(self.label_low)
        vbox.addWidget(self.slider_mid)
        vbox.addWidget(self.label_mid)
        vbox.addWidget(self.slider_high)
        vbox.addWidget(self.label_high)
        vbox.addWidget(self.button_apply)
        self.setLayout(vbox)
        
        # Configurar la ventana principal
        self.setWindowTitle("Ecualizador de audio")
        self.setGeometry(100, 100, 300, 200)
        self.show()
        
    def update_label_low(self, value):
        # Actualizar la etiqueta para la banda de frecuencia baja
        self.label_low.setText(f"Baja frecuencia: {value} dB")
        
    def update_label_mid(self, value):
        # Actualizar la etiqueta para la banda de frecuencia media
        self.label_mid.setText(f"Frecuencia media: {value} dB")
        
    def update_label_high(self, value):
        # Actualizar la etiqueta para la banda de frecuencia alta
        self.label_high.setText(f"Alta frecuencia: {value} dB")
        
    def apply_eq(self):
        # Aplicar la ecualización a la señal de audio
        eq_low = self.slider_low.value()
        eq_mid = self.slider_mid.value()
        eq_high = self.slider_high.value()
        
        audio_eq = self.audio_file \
            .equalize( \
                bands=[(0,200, eq_low), \
                   (200, 2000, eq_mid), \
                   (2000, None, eq_high)])
                   
        # Reproducir la señal de audio ecualizada
        play(audio_eq)
if __name__ == "__main__":
    app = QApplication([])
    window = MainWindow()
    app.exec_()